# 2021-08-17 Adapting the framework implementation to the general model formulation for SCPaper

* Went over all of the steps in the "more constrained general formulation" in the SCPaper README, makin gsure that the implemenation and notation in `sc_senario.py` is in line with these. Removed some old, extraneous stuff. Tested the following runs, confirming that I am getting back the same results as in the 2021-06-01 diary entry:
    * Baseline kinematics, baseline model.
    * Baseline kinematics, baseline model + `oBEv`
    * Baseline kinematics, baseline model + `oBEo`